In [2]:
import pandas as pd
import jsonlines

# This is for the Cancer Exclusion NER Correct Task

In [5]:
cts = pd.read_csv("/home/vs428/project/Moore_data/all_moore_cts.csv")

In [3]:
data = pd.read_csv("/home/vs428/project/Moore_models/nlp_cancer_samp.csv")

In [20]:
cts[~cts['KL_ID'].isin(data['Unique study ID KL'])].sample(1000).to_csv("/home/vs428/project/Moore_models/nlp_cancer_excl_nercorrect.csv", index=False)

# This is for the Followup textcat Correct task

In [5]:
import sklearn

In [2]:
data = pd.read_csv("/home/vs428/project/Moore_models/Task_4_textcat_input.csv")

In [3]:
all_notes = pd.read_csv("/home/vs428/project/Moore_data/all_moore_cts_cancer_excl_preds.csv")

In [14]:
print(all_notes['Nodule_model'].value_counts())
print(all_notes["Cancer_on_CT_NLP_rec"].value_counts())

follow_up_pipeline_notes = all_notes[(all_notes['Nodule_model'] == "Nodule") & 
         (~all_notes["Cancer_on_CT_NLP_rec"])]


No Nodule    16543
Nodule       10002
Name: Nodule_model, dtype: int64
False    26255
True       290
Name: Cancer_on_CT_NLP_rec, dtype: int64


In [22]:
follow_up_pipeline_notes_filter_anns = follow_up_pipeline_notes[~follow_up_pipeline_notes['KL_ID'].isin(data['study_id'].unique())].sample(1000)


In [24]:
follow_up_pipeline_notes_filter_anns

,KL_ID,Age,person_id,CT_Text,Adjudicated_Recommendation,Model_Recommendation,Nodule_model,Measurement_date,CT_type,Cancer_on_CT_NLP_score,Cancer_on_CT_NLP_rec
26224,KL_10047,81,RD_14005,Study: CTA CHEST ABDOMEN W AND/OR WO IV CONTRA...,NO_FOLLOWUP,HARD_FOLLOWUP,Nodule,2015-12-09 03:11:00,Custom: IMG2864 CTA CHEST ABDOMEN W AND/OR WO...,0.003100,False
3011,KL_828,40,RD_522,"CT CHEST ABDOMEN PELVIS W IV CONTRAST, CT 2D R...",HARD_FOLLOWUP,HARD_FOLLOWUP,Nodule,2017-12-17 10:13:00,Custom: IMG1428 CT CHEST ABDOMEN PELVIS W IV ...,0.003074,False
6102,KL_3115,32,RD_7667,"CT CHEST, ABDOMEN, AND PELVIS WITH INTRAVENOUS...",NO_FOLLOWUP,NO_FOLLOWUP,Nodule,2017-07-04 16:09:00,Custom: IMG1428 CT CHEST ABDOMEN PELVIS W IV ...,0.003098,False
2199,KL_954,34,RD_4122,CT CHEST ABDOMEN PELVIS W IV CONTRAST INDICAT...,CONDITIONAL_FOLLOWUP,CONDITIONAL_FOLLOWUP,Nodule,2019-04-21 10:31:00,Custom: IMG1428 CT CHEST ABDOMEN PELVIS W IV ...,0.003064,False
6937,KL_15792,56,RD_8841,"CT PULMONARY ANGIOGRAM INDICATION: Cough, ta...",NO_FOLLOWUP,NO_FOLLOWUP,Nodule,2017-02-03 20:51:00,Custom: IMG3904 CTA CHEST (PE) W IV CONTRAST,0.003065,False
...,...,...,...,...,...,...,...,...,...,...,...
18162,KL_19715,81,RD_8906,"CTA CHEST WITH CONTRAST, PULMONARY EMBOLISM PR...",NO_FOLLOWUP,NO_FOLLOWUP,Nodule,2015-12-27 19:34:00,Custom: IMG3904 CTA CHEST (PE) W IV CONTRAST,0.003064,False
8805,KL_4227,56,RD_10796,CTA CHEST (PE) W IV CONTRAST INDICATION: his...,NO_FOLLOWUP,NO_FOLLOWUP,Nodule,2020-05-13 00:40:00,Custom: IMG3904 CTA CHEST (PE) W IV CONTRAST,0.003040,False
14321,KL_6238,57,RD_4044,CTA CHEST (PE) W IV CONTRAST INDICATION: Pulmo...,NO_FOLLOWUP,NO_FOLLOWUP,Nodule,2021-11-11 18:49:00,Custom: IMG3904 CTA CHEST (PE) W IV CONTRAST,0.003088,False
307,KL_9900,37,RD_16456,"CLINICAL INFORMATION: CP or SOB, high pretest ...",NO_FOLLOWUP,NO_FOLLOWUP,Nodule,2018-07-22 15:43:00,Custom: IMG3904 CTA CHEST (PE) W IV CONTRAST,0.003080,False


In [23]:
follow_up_pipeline_notes_filter_anns.columns

Index(['KL_ID', 'Age', 'person_id', 'CT_Text', 'Adjudicated_Recommendation',
       'Model_Recommendation', 'Nodule_model', 'Measurement_date', 'CT_type',
       'Cancer_on_CT_NLP_score', 'Cancer_on_CT_NLP_rec'],
      dtype='object')

In [35]:
follow_up_pipeline_notes_filter_anns[['KL_ID', "person_id", "CT_Text", "Measurement_date", 
                                      "Nodule_model", "Cancer_on_CT_NLP_rec"]].sample(frac=1).rename({"CT_Text":"text"}, axis=1).to_csv("/home/vs428/project/Moore_data/all_moore_cts_followup_ann_v2.csv", index=False)



# Create Joint Training Data for Cancer Exclusion from Annotations + Reviewed

In [2]:
import jsonlines
from collections import Counter

In [3]:
def merge_ner_manual_correct(ner_manual_data, ner_correct_data):
    '''Merge jsonl files from prodigy's ner.manual and ner.correct. All we're doing is checking input hashes and choosing 
    the ner.correct one if it exists, otherwise dropping it. Not sure if prodigy has a function for this or not (db.merge seems different)
    '''
    
    merged = []
    # print(ner_correct_data[0].keys())
    # print(ner_correct_data[0]["_input_hash"])
    ner_correct_hashes = {data["_input_hash"]:data for data in ner_correct_data}
    print(len(ner_correct_hashes.keys()))
    print(len(ner_correct_data))
    assert len(ner_correct_hashes.keys()) == len(ner_correct_data), "Not unique hashes?"
    
    for ner_man_data in ner_manual_data:
        if ner_man_data['_input_hash'] in ner_correct_hashes.keys():
            merged.append(ner_correct_hashes[ner_man_data['_input_hash']])
        else:
            merged.append(ner_man_data)
    
    return merged
    

In [4]:
data = []
with jsonlines.open("/home/vs428/project/Moore_data/moore_cancer_exclusion_v2.jsonl") as reader:
    for line in reader:
        data.append(line)
    
reviewed_data = []
with jsonlines.open("/home/vs428/project/Moore_data/moore_cancer_exclusion_REVIEW_v2.jsonl") as reader:
    for line in reader:
        reviewed_data.append(line)
    
        
        

In [5]:
# drop rows if they contains vs428 (I annotated them for some reason?)
data = [x for x in data if "vs428" not in x["_session_id"]]
reviewed_data = [x for x in reviewed_data if "vs428" not in x["_session_id"]]

In [6]:
# out = merge_ner_manual_correct([data[0]], [reviewed_data[0]])
out = merge_ner_manual_correct(data, reviewed_data)

477
477


In [9]:
with jsonlines.open("/home/vs428/project/Moore_data/moore_cancer_exclusion_MERGED_v2.jsonl", "w") as writer:
    writer.write_all(out)

# Other

In [19]:
# anns = []
# import jsonlines
# with jsonlines.open("/home/vs428/project/Moore_data/followup_pipeline_v1.jsonl") as reader:
#     for x in reader:
#         anns.append(x)

# def get_skipped_anns(data):
#     '''inputs a jsonl file and extracts all the annotations that were skipped
#     '''
#     return [ann for ann in data if ann['answer'] == "ignore"]        

# z = get_skipped_anns(anns)

# with open("/home/vs428/project/Moore_data/followup_pipeline_v1_skipped_ct_fn.txt", "w") as f:
#     for ann in z:
#         f.write(ann['text'])
#         f.write("\n\n")

# Create Training Data (Impression Only)

In [43]:
import pandas as pd

In [44]:
data = pd.read_csv("/home/vs428/project/current cancer flags 6_23_2022.csv")

In [45]:
all_raw_text = pd.read_csv("/home/vs428/project/ct_ed.tsv", sep="\t")

In [46]:
data.columns

Index(['Unnamed: 0', 'person_id', 'location_id', 'birth_datetime', 'age_years',
       'gender', 'ethnicity', 'race', 'visit_start_datetime',
       'visit_end_datetime', 'visit_occurrence_id', 'value_source_value',
       'measurement_datetime', 'measurement_source_value', 'study_id', 'zip',
       'plan_source_value', 'phenotype_start_date', 'phenotype_end_date',
       'payer_plan_period_start_date', 'study_hash', 'dist',
       'insurance_category', 'person_ids_study', 'PAT_ENC_CSN_ID', 'MRN_ID',
       'NLP_OUT', 'future_cancer', 'prior_cancer', 'future_cancer_names',
       'prior_cancer_names'],
      dtype='object')

In [47]:
all_raw_text.columns

Index(['PAT_ENC_CSN_ID', 'PAT_MRN_ID', 'visit_occurrence_id', 'person_id',
       'OrderingEncounterKey', 'OrderType', 'Narrative', 'Impression',
       'OrderingInstant', 'Name', 'Code'],
      dtype='object')

In [48]:
data.shape, all_raw_text.shape

((26545, 31), (77466, 11))

In [49]:
data['measurement_datetime'] = pd.to_datetime(data['measurement_datetime'])
all_raw_text["OrderingInstant"] = pd.to_datetime(all_raw_text["OrderingInstant"])

In [50]:
data['measurement_hour'] = data['measurement_datetime'].dt.hour
all_raw_text["Orderinghour"] = all_raw_text["OrderingInstant"].dt.hour

In [52]:
all_raw_text.columns

Index(['PAT_ENC_CSN_ID', 'PAT_MRN_ID', 'visit_occurrence_id', 'person_id',
       'OrderingEncounterKey', 'OrderType', 'Narrative', 'Impression',
       'OrderingInstant', 'Name', 'Code', 'Orderinghour'],
      dtype='object')

In [74]:
merged  = data.merge(all_raw_text[['PAT_ENC_CSN_ID', 'OrderingInstant', "Orderinghour", 'Narrative', 'Impression']],
           left_on=["PAT_ENC_CSN_ID"],#, "measurement_hour"],
           right_on=["PAT_ENC_CSN_ID"],#, "Orderinghour"],
           how="left",
           indicator=True)

#### merged['_merge'].value_counts()

In [61]:
# set(data['PAT_ENC_CSN_ID'].unique()) - set(all_raw_text['PAT_ENC_CSN_ID'].unique())

In [66]:
data[data['PAT_ENC_CSN_ID'] == 148520964.0]
# all_raw_text[all_raw_text['PAT_ENC_CSN_ID'] == 148520964]

,Unnamed: 0,person_id,location_id,birth_datetime,age_years,gender,ethnicity,race,visit_start_datetime,visit_end_datetime,...,insurance_category,person_ids_study,PAT_ENC_CSN_ID,MRN_ID,NLP_OUT,future_cancer,prior_cancer,future_cancer_names,prior_cancer_names,measurement_hour
7331,7331,13987903,ca438e4d2a9af1d3195bcb9cc61cc092cf6011d1d3f41e...,1/25/58,59,FEMALE,Not Hispanic or Latino,White,3/30/17 22:56,4/6/17 12:47,...,Commercial,RD_8298,148520964.0,MR1034275,"{\n ""0"": {\n ""word"": ""pleura"",\n ""label...",True,True,"['Malignant neoplasm of main bronchus', 'Malig...","['History of malignant neoplasm of bronchus', ...",9


In [54]:
data['PAT_ENC_CSN_ID'].value_counts()

181246545.0    6
203494512.0    6
161514536.0    5
95782058.0     5
202362471.0    5
              ..
225400642.0    1
247733970.0    1
210011020.0    1
229247197.0    1
124199208.0    1
Name: PAT_ENC_CSN_ID, Length: 26052, dtype: int64

In [55]:
merged.shape

(4414, 36)

In [56]:
merged['PAT_ENC_CSN_ID'].value_counts()

161514536.0    3
163671797.0    2
126161129.0    2
244519838.0    2
267125813.0    2
              ..
169531057.0    1
109885769.0    1
245156107.0    1
208036612.0    1
138064351.0    1
Name: PAT_ENC_CSN_ID, Length: 4401, dtype: int64

# Create Training Data (Impression Only) - From Other Data

In [7]:
anns = []
with jsonlines.open("/home/vs428/project/Moore_data/followup_pipeline_v2.jsonl") as reader:
    for line in reader:
        anns.append(line)

In [8]:
anns[0]['text'][-1:]

'D'

In [9]:
print(anns[3]['text'])

. Likely subacute fracture of spinous process of T11. 4.  Moderately severe upper lobe predominant centrilobular emphysema. Indeterminate solid pulmonary nodule measuring less than 6 mm. In a low-risk patient with a solid nodule <6 mm, recommend no follow-up. In a high-risk patient, CT at 12 months is optional with stronger consideration if there is suspicious nodule morphology and/or upper lobe location. 5.  Bilateral axillary lymphadenopathy of unclear etiology, consider correlation with mammogram and/or dedicated ultrasound.  The above findings were discussed with Dr. Verghese at 8:00 PM on 3/1/2019  Reported By:  Sarvenaz Pourjabbar, MD  Reported And Signed By: Krithica Kaliannan,


In [12]:
impressions = []
for ann in anns:
    try:
        impression = ann
        if re.search(r"IMPRESSION:|Impression:", ann['text']):
            idx_start = re.search(r"IMPRESSION:|Impression:", ann['text']).start()
        else:
            idx_start = 0
        impression['text'] = ann['text'][idx_start:]
        impressions.append(impression)
        
    except ValueError:
        impressions.append(ann)
        

In [13]:
for imp in impressions:
    if "versions" in imp:
        imp.pop("versions")

In [14]:
len(impressions)

1001

In [16]:
impressions[:10]

[{'text': "IMPRESSION:   1. No evidence of pulmonary embolism or acute thoracic pathology. [ 2. Redemonstration of lucent lesions throughout the spine, ribs, and sternum consistent with the patient's known history of multiple myeloma, which appear stable since the examination on 3/14/2019  3. Indeterminate renal lesions in the right kidney, including a lesion measuring up to 3.9 cm, are identified and further characterization with multiphase MRI is recommended on an outpatient basis   Report Initiated By:  Andrew Kuhn, MD  A critical alert Blue ED Follow Up message has been communicated to this patient's provider via the Nuance Actionable Findings application on 10/25/2019 7:20 AM, Message ID 3586180.  Reported And Signed By: Syed Jamal Bokhari, MD",
  '_input_hash': -1983226661,
  '_task_hash': -941005989,
  'options': [{'id': 'NO_FOLLOWUP', 'text': 'NO_FOLLOWUP'},
   {'id': 'HARD_FOLLOWUP', 'text': 'HARD_FOLLOWUP'},
   {'id': 'CONDITIONAL_FOLLOWUP', 'text': 'CONDITIONAL_FOLLOWUP'}],


In [17]:
with jsonlines.open("/home/vs428/project/Moore_data/followup_pipeline_v2_impressions.jsonl", "w") as writer:
    writer.write_all(impressions)

# Merge in Training Data (Impression Only) - From Other Data - New 100 as of 1/11/2023

In [2]:
import jsonlines
import re

In [41]:
anns = []
with jsonlines.open("/home/vs428/project/Moore_data/followup_pipeline_v2.jsonl") as reader:
    for line in reader:
        anns.append(line)

In [42]:
len(anns)

1001

In [43]:
newanns = []
with jsonlines.open("/home/vs428/project/Moore_data/followup_pipeline_v2_600-rdenkewicz.jsonl") as reader:
    for line in reader:
        newanns.append(line)

In [44]:
len(newanns)

600

In [54]:
from collections import Counter
counter = Counter()
ryan_old = []
for ann in anns:
    if ann['_annotator_id'].startswith('ct_images_75_25_500_TASK4'):
        # this is the initial set of 500 original ones adjudicated by Chris or someone?
        counter['ct_images_75_25_500_TASK4'] += 1
    elif ann['_annotator_id'].startswith("followup_pipeline_v1-rdenkewicz"):
        # this is the set of 500 more that just Ryan did
        counter['followup_pipeline_v1-rdenkewicz'] += 1        
        ryan_old.append(ann)
    else:
        print(ann['_annotator_id'])

In [55]:
counter

Counter({'ct_images_75_25_500_TASK4': 501,
         'followup_pipeline_v1-rdenkewicz': 500})

In [56]:
# we're checking the same thing for the new set that Ryan annotated (extra 100)
# so we have 500 from before + 100 new ones
from collections import Counter
counter2 = Counter()
for ann in newanns:
    if ann['_annotator_id'].startswith('ct_images_75_25_500_TASK4'):
        counter2['ct_images_75_25_500_TASK4'] += 1
    elif ann['_annotator_id'].startswith("followup_pipeline_v1-rdenkewicz"):
        counter2['followup_pipeline_v1-rdenkewicz'] += 1        
    else:
        print(ann['_annotator_id'])
counter2

Counter({'followup_pipeline_v1-rdenkewicz': 600})

In [57]:
# this is the set of ryan's old 500 annotation hashes
old_ryan_hashes = set([ann['_input_hash'] for ann in others])

In [73]:
len(set([ann['_input_hash'] for ann in newanns]))

600

In [60]:
counter3 = Counter()
for ann in newanns:
    if ann['_input_hash'] in old_ryan_hashes and ann['_annotator_id'].startswith("followup_pipeline_v1-rdenkewicz"):
        counter3['yes']  += 1
    else:
        counter3['no']  += 1

In [61]:
# we confirm that we have 500 old ones and 100 new ones
counter3

Counter({'yes': 500, 'no': 100})

In [63]:
# we merge in the 100 new ones here with the 1000 that already exist to get 1101
new_ryan_anns = []
for ann in newanns:
    if ann['_input_hash'] not in old_ryan_hashes:
        new_ryan_anns.append(ann)

In [65]:
merged_anns = anns + new_ryan_anns

In [72]:
len(anns) , len(new_ryan_anns)
# 1001 + 100

(1001, 100)

In [69]:
len(set([ann['_input_hash'] for ann in merged_anns]))

1099

In [74]:
len(merged_anns)
# we lose two here for some reason but who cares
assert len(set([ann['_input_hash'] for ann in merged_anns])) == 1101

AssertionError: 

In [76]:
with jsonlines.open("/home/vs428/project/Moore_data/followup_pipeline_v3.jsonl", "w") as writer:
    writer.write_all(merged_anns)

In [77]:
# Convert this into IMPRESSIONS only

impressions = []
for ann in merged_anns:
    try:
        impression = ann
        if re.search(r"IMPRESSION:|Impression:", ann['text']):
            idx_start = re.search(r"IMPRESSION:|Impression:", ann['text']).start()
        else:
            idx_start = 0
        impression['text'] = ann['text'][idx_start:]
        impressions.append(impression)
        
    except ValueError:
        impressions.append(ann)
        

In [78]:
for imp in impressions:
    if "versions" in imp:
        imp.pop("versions")

In [79]:
len(impressions)

1101

In [80]:
with jsonlines.open("/home/vs428/project/Moore_data/followup_pipeline_v3_impressions.jsonl", "w") as writer:
    writer.write_all(impressions)

In [81]:
# now we run data-to-spacy from prodigy
# python -m spacy init fill-config base_followup_weighted_config.cfg followup_weighted_config.cfg --code  ./Weighted_TextCategorizer.py


In [89]:
# let's check the docbins we created

In [82]:
import pandas as pd
import spacy
from spacy.tokens import DocBin

In [86]:
docbin = DocBin().from_disk("/home/vs428/project/Moore_data/followup_impression_data_1_11_2023/train.spacy")
dev_docbin = DocBin().from_disk("/home/vs428/project/Moore_data/followup_impression_data_1_11_2023/dev.spacy")

In [88]:
# we only have 1007 because of all the ones that were dropped
len(docbin), len(dev_docbin), len(docbin) + len(dev_docbin)

(856, 151, 1007)

In [3]:
anns = []
with jsonlines.open("/home/vs428/project/Moore_data/followup_pipeline_v3_impressions.jsonl") as reader:
    for line in reader:
        anns.append(line)

In [10]:
from collections import Counter
Counter([ann['accept'][0] for ann in anns if ann['accept']])

Counter({'NO_FOLLOWUP': 600,
         'CONDITIONAL_FOLLOWUP': 222,
         'HARD_FOLLOWUP': 189})

# Convert DocBin into CSV

In [82]:
import pandas as pd
import spacy
from spacy.tokens import DocBin

In [25]:
docbin = DocBin().from_disk("/home/vs428/project/Moore_data/followup_impression_data/dev.spacy")


In [ ]:
nlp = spacy.load("en_core_web_trf")

In [57]:
docs = docbin.get_docs(nlp.vocab)

In [58]:
docs_list = []
for doc in docs:
    docs_list.append({
        "text": doc.text,
        "label": max(doc.cats, key=doc.cats.get)
    })

In [59]:
len(df_docs), len(docbin)

(138, 138)

In [60]:
df_docs = pd.DataFrame.from_records(docs_list)

In [74]:
df_docs.to_csv("/home/vs428/project/Moore_data/followup_impression_data/dev.csv")

In [71]:
iscout_input = [str(idx) + " ~" + row + "\n" for idx, row in df_docs['text'].iteritems()]

/tmp/ipykernel_137134/2760701896.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  iscout_input = [str(idx) + " ~" + row + "\n" for idx, row in df_docs['text'].iteritems()]


In [73]:
with open("/home/vs428/Documents/Moore/iScout/iscout_ct_impressions_only.txt", "w") as f:
    for x in iscout_input:
        f.write(x)